In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import sklearn.cluster as sc

# import sklearn.datasets as sd
# import sklearn.feature_extraction.text as ft
# import sklearn.model_selection as ms
# import sklearn.linear_model as lm
# import sklearn.tree as st
# import sklearn.ensemble as se
# import sklearn.naive_bayes as nb
# import sklearn.svm as svm
import sklearn.metrics as sm


In [19]:
ratings = pd.read_json("ratings.json")
ratings

,John Carson,Michelle Peterson,William Reynolds,Jillian Hobart,Melissa Jones,Alex Roberts,Michael Henry
Inception,2.5,3.0,2.5,NaN,3,3.0,NaN
Pulp Fiction,3.5,3.5,3.0,3.5,4,4.0,4.5
Anger Management,3.0,1.5,NaN,3.0,2,NaN,NaN
Fracture,3.5,5.0,3.5,4.0,3,5.0,4.0
Serendipity,2.5,3.5,NaN,2.5,2,3.5,1.0
Jerry Maguire,3.0,3.0,4.0,4.5,3,3.0,NaN


In [24]:
# 计算用户与用户之间的相似度矩阵
simmat = ratings.corr()
simmat



,John Carson,Michelle Peterson,William Reynolds,Jillian Hobart,Melissa Jones,Alex Roberts,Michael Henry
John Carson,1.000000,0.396059,0.404520,0.566947,0.594089,0.747018,0.991241
Michelle Peterson,0.396059,1.000000,0.204598,0.314970,0.411765,0.963796,0.381246
William Reynolds,0.404520,0.204598,1.000000,1.000000,-0.258199,0.134840,-1.000000
Jillian Hobart,0.566947,0.314970,1.000000,1.000000,0.566947,0.028571,0.893405
Melissa Jones,0.594089,0.411765,-0.258199,0.566947,1.000000,0.211289,0.924473
Alex Roberts,0.747018,0.963796,0.134840,0.028571,0.211289,1.000000,0.662849
Michael Henry,0.991241,0.381246,-1.000000,0.893405,0.924473,0.662849,1.000000


# 基于simmat相似度矩阵，使用UBCF（基于用户相似度的）算法实现简单推荐引擎

In [21]:
# 获取登录用户
login_usr = "Michael Henry"
# 寻找登录用户的相似用户：去掉自己、去掉反相关的用户
sim_usrs = simmat[login_usr]
sim_usrs = sim_usrs.drop(login_usr)
sim_usrs = sim_usrs[sim_usrs>0]



# 遍历所有相似用户
# 找个合适的数据结构，存储数据列表，还有所有人对每一步推荐电影的打分
recomment_list={}
for sim_usr,sim_score in sim_usrs.items():
    movies = ratings[sim_usr]
    for movie,score in movies.dropna().items():
        if np.isnan(ratings[login_usr][movie]):
            if movie not in recomment_list.keys():
                recomment_list[movie]=[]
            recomment_list[movie].append((score,sim_score))
            

recomment_list

{'Inception': [(2.5, 0.9912407071619305),
  (3.0, 0.3812464258315117),
  (3, 0.924473451641905),
  (3.0, 0.6628489803598702)],
 'Anger Management': [(3.0, 0.9912407071619305),
  (1.5, 0.3812464258315117),
  (3.0, 0.8934051474415642),
  (2, 0.924473451641905)],
 'Jerry Maguire': [(3.0, 0.9912407071619305),
  (3.0, 0.3812464258315117),
  (4.5, 0.8934051474415642),
  (3, 0.924473451641905),
  (3.0, 0.6628489803598702)]}

In [22]:
#对电影列表排序
def func(item):
    s=0
    for d in item[1]:
        s += d[0]*d[1]
    
    s = s/len(item[1])
    print(item[0],"综合推荐得分为：",s)
    return s


r = sorted(recomment_list.items(),key=func,reverse=True)

print(np.array(r)[:,0])


Inception 综合推荐得分为： 2.0959520853511715
Anger Management 综合推荐得分为： 2.0186885264603904
Jerry Maguire 综合推荐得分为： 2.579950371694538
['Jerry Maguire' 'Inception' 'Anger Management']


C:\Users\WJ\AppData\Local\Temp/ipykernel_3396/527907032.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(r)[:,0])
